In [40]:
using BenchmarkTools
using StaticArrays
using Random
using Plots
using LinearAlgebra

### Benchmarking Code

Julia is just in time (JIT) compiled, so the first time you run code there will be a penalty to first compile the code before it can be run. We can use the `@time` macro to see this effect. (If you run the cell more than once the effect will go away since it has already been compiled!)

In [41]:
x = rand(1000);
function sum_global()
    s = 0.0
    for i in x
        s += i
    end
    return s
end;

In [42]:
@time sum_global();
@time sum_global();

  0.000235 seconds (3.49 k allocations: 70.156 KiB)
  0.000131 seconds (3.49 k allocations: 70.156 KiB)


The `@time` macro is convenient, but only executes the function once. We can use the `BenchmarkTools.jl` library to investigate the performance of our function with multiple trials. The `@benchmark` macro will run 10,000 trials or (roughly) 5 seconds whichever comes first. (These can be configured). There is also the `@btime` macro which runs the same number of trials as `@benchmark` but has output similar to `@time`.

In [43]:
@benchmark sum_global()

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  74.900 μs …  1.511 ms  ┊ GC (min … max): 0.00% … 91.95%
 Time  (median):     77.900 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   81.754 μs ± 54.465 μs  ┊ GC (mean ± σ):  2.50% ±  3.57%

  ▂▅▅▅▇█▆▂             ▁▁▁      ▁▁                       ▁▁▁  ▂
  ████████▇▆▆▆▆▅▅▅▅▆▅▆████▇▅▆▆▇▇██▇▅▄▅▄▃▄▁▄▃▁▃▃▄▄▆▇▆▆▆███████ █
  74.9 μs      Histogram: log(frequency) by time       109 μs <

 Memory estimate: 70.16 KiB, allocs estimate: 3490.

If your function takes parameters, be sure to "interpolate" them with a `$` when using `@benchmark` this tells `BenchmarkTools.jl` to ignore the allocation and time required to allocate the parameters.

In [44]:
# Parameter is Interpolated
@btime inv($(rand(6,6)));
# No Interpolation
@btime inv(rand(6,6));

  1.060 μs (4 allocations: 3.59 KiB)
  1.360 μs (5 allocations: 3.95 KiB)


### Reducing Memory Allocations
One of the easiest ways to speed up your code is by removing all unnecessary memory allocations. In Python the end-user is not able to manage memory easily and the interprer or library (e.g. numpy) is trusted to handle things.

Julia can also operate like Python, where we just trust the compiler to "do the right thing". This is good for prototyping, but can result in inefficient code. Below we will look at the basics of memory allocations and how to reduce them.

-----------------------------------------------
Lets start by allocating a small array of 10,000 Float64s. Each Float64 is 8 bytes (64 bits), so the total size should be 80,000 bytes or 80kB (78.125 kiB). Note there will be a slight overhead for some internal machinery associated with each allocation (e.g. a pointer to the array).

In [45]:
@time ones(Float64, 100, 100);

  0.000040 seconds (2 allocations: 78.172 KiB)


In Julia all arrays are `heap` allocated since they are re-sizeable. This just means that the array is further away from the CPU and it can be slow(er) to allocate and access. The first strategy to mitigate this penalty is to simply allocate memory as little as possible. To check this we can use our `BenchmarkTools.jl` library to understand the allocations in our function.

In [46]:
function allocates_alot()
    res = 0.0
    for i in 1:100
        x = rand(100,100) # ALLOCATION EVERY LOOP ITERATION
        res += sum(x)
    end
    return res
end

function allocates_once()
    res = 0.0
    storage = zeros(100,100)
    for i in 1:100
        #The ! point means it mutates the input
        # In this case it over writes storage with new random values
        rand!(storage)
        res += sum(storage)
    end
    return res
end


allocates_once (generic function with 1 method)

Using the `@btime` macro we can measure the performace uplift and the number of allocations from each function. You'll find that the function which allocates a new array each loop iteration will be slower and allocate far more memory.

In [47]:
@btime allocates_alot() samples = 1000;
@btime allocates_once() samples = 1000;

  845.800 μs (200 allocations: 7.63 MiB)
  409.200 μs (2 allocations: 78.17 KiB)


A more subtle, yet common place where allocations occur is array slices (e.g. `arr[1:10]`). In Julia these slices allocate a new array by default. We can get around this by telling Julia that we only want to read the data inside the arr. To do this we use an `array view` which can be acheive with the `view(...)` function or the `@views` macro.

In [48]:
function slices_allocate(x)
    N = Int(length(x) / 2)
    res = sum(x[1:N])
    return res
end

function view_slices(x)
    N = Int(length(x) / 2)
    #The view function takes the array
    # and the indicies you want a view of.
    res = sum(view(x, 1:N))
    return res
end

function view_macro_slices(x)
    N = Int(length(x) / 2)
    # The @views macro tells Julia all array slices in
    # this line should be array views.
    @views res = sum(x[1:N])
    return res
end

view_macro_slices (generic function with 1 method)

Again we see a large speed increase and the number of allocations drop!

In [49]:
x = rand(100000)
@btime slices_allocate(x) samples = 1000;
@btime view_slices(x) samples = 1000;
@btime view_macro_slices(x) samples = 1000;

  57.700 μs (3 allocations: 390.69 KiB)
  5.317 μs (1 allocation: 16 bytes)
  5.333 μs (1 allocation: 16 bytes)


### StaticArrays
Earlier we mentioned that Julia arrays are `heap` allocated because they have variable length (i.e., you can append to them). If you know the length of your array beforehand, you can use the `StaticArrays.jl` library to `stack` allocate your data (if its small enough).

`StaticArrays.jl` has many types you can use, but the main types are `SVector` for 1D arrays, `SMatrix` for 2D arrays, and `SArray` for n-dimensional arrays (all types have equivalent macros e.g., `@SMatrix`).

To initialize a NxN `SMatrix` we must provide the size to the constructor: `SMatrix{N,N}(...)`

In [50]:
m = SMatrix{25,25}(rand(25,25));

A common pattern in atomistic simulation packages is to have an array of `SVectors` which represent atomic properties, like their position. Storing the data for each atom as an SVector can be much faster than using the default Julia array.

In [51]:
N_atoms = 1000
static_positions = [SVector{3}(rand(3)) for i in 1:N_atoms];
dynamic_positions = [rand(3) for i in 1:N_atoms];

function distance_matrix(positions)
    N = length(positions)
    dists = zeros(N,N)
    for i in 1:N
        for j in i:N
            dists[i,j] = norm(positions[i] - positions[j])
            dists[j,i] = dists[i,j]
        end
    end
    return dists
end


distance_matrix (generic function with 1 method)

In [52]:
println("Static Vector Benchmark:")
@benchmark distance_matrix(static_positions)

Static Vector Benchmark:


BenchmarkTools.Trial: 1402 samples with 1 evaluation.
 Range (min … max):  2.898 ms … 13.290 ms  ┊ GC (min … max):  0.00% … 33.18%
 Time  (median):     3.024 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.559 ms ±  1.227 ms  ┊ GC (mean ± σ):  15.11% ± 19.44%

   █▅                                                         
  ▅██▆▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▂▂▂▂▂▄▄▃▂ ▂
  2.9 ms         Histogram: frequency by time         6.3 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

In [53]:
println("Dynamic Array Benchmark:")
@benchmark distance_matrix(dynamic_positions)

Dynamic Array Benchmark:


BenchmarkTools.Trial: 156 samples with 1 evaluation.
 Range (min … max):  30.954 ms … 50.233 ms  ┊ GC (min … max): 6.95% … 9.65%
 Time  (median):     31.861 ms              ┊ GC (median):    7.91%
 Time  (mean ± σ):   32.131 ms ±  1.785 ms  ┊ GC (mean ± σ):  7.96% ± 0.37%

        ▄██▆▃▃                                                 
  ▅▁▁▁▁▁██████▇▅▁▇▁▅▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▅
  31 ms        Histogram: log(frequency) by time      37.2 ms <

 Memory estimate: 45.81 MiB, allocs estimate: 500502.

### Type Stability
Julia will do its best to infer the data types of your variables, but if the compiler cannot infer the type of all your variables this leads to type instability and will slow down your code. We can check type stability with the `@code_warntype` macro. For today's purposes if this macro returns any red text then your code has unstable types.

In the example below, the functions are equivalent except one relies on a gloabl variable and the other takes that variable as a parameter. Because of the global variable, the compiler must assume that the variables inside the `unstable` function have `Any` type. Whereas in the type stable code the compiler can create a specialized version of the `stable` function for the specific type you passed into it. This is not the only unstable pattern, so if you really care about speed check your code with `@code_warntype`

In [60]:
global_x = rand(Float64, 1000)
function unstable()
    s = 0
    for val in global_x
      s = s + val
    end
    return s
end

function stable(x)
  s = zero(eltype(x))
  for val in x
     s = s + val
  end
  return s
end

stable (generic function with 1 method)

In [61]:
@code_warntype unstable()

MethodInstance for unstable()
  from unstable() @ Main c:\Users\ejmei\repos\MolSSI_workshop\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X34sZmlsZQ==.jl:2
Arguments
  #self#::Core.Const(unstable)
Locals
  @_2::Any
  s::Any
  val::Any
Body::Any
1 ─       (s = 0)
│   %2  = Main.global_x::Any
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_2::Any
│         (val = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Any
│         (s = s + val)
│         (@_2 = Base.iterate(%2, %9))
│   %12 = (@_2 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return s



In [62]:
@code_warntype stable(global_x)

MethodInstance for stable(::Vector{Float64})
  from stable(x) @ Main c:\Users\ejmei\repos\MolSSI_workshop\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X34sZmlsZQ==.jl:10
Arguments
  #self#::Core.Const(stable)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  val::Float64
Body::Float64
1 ─ %1  = Main.eltype(x)::Core.Const(Float64)
│         (s = Main.zero(%1))
│   %3  = x::Vector{Float64}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Float64, Int64}
│         (val = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│         (s = s + val)
│         (@_3 = Base.iterate(%3, %10))
│   %13 = (@_3 === nothing)::Bool
│   %14 = Base.not_int(%13)::Bool
└──       goto #4 if not %14
3 ─       goto #2
4 ┄       return s



In [63]:
@btime unstable();

  76.000 μs (3490 allocations: 70.16 KiB)


In [64]:
@btime stable($global_x);

  930.000 ns (0 allocations: 0 bytes)
